In [1]:
!pip install anthropic
!pip install fpdf
!pip install python-docx



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import re
import json
from collections import Counter
from datetime import datetime

# For AI parsing
try:
    from anthropic import Anthropic
    AI_AVAILABLE = True
except:
    AI_AVAILABLE = False
    print("⚠️ Anthropic not installed. Run: !pip install anthropic")

# For resume generation
try:
    from fpdf import FPDF
    PDF_AVAILABLE = True
except:
    PDF_AVAILABLE = False
    print("⚠️ FPDF not installed. Run: !pip install fpdf")

try:
    from docx import Document
    from docx.shared import Pt, RGBColor
    from docx.enum.text import WD_ALIGN_PARAGRAPH
    DOCX_AVAILABLE = True
except:
    DOCX_AVAILABLE = False
    print("⚠️ python-docx not installed. Run: !pip install python-docx")


class AIJobParser:
    def __init__(self, api_key=None):
        self.api_key = api_key
        if api_key and AI_AVAILABLE:
            self.client = Anthropic(api_key=api_key)

    def parse_with_ai(self, jd_text):
        """Use Claude AI to parse job description intelligently"""
        if not self.api_key or not AI_AVAILABLE:
            print("⚠️ AI parsing not available. Use basic parser instead.")
            return None

        prompt = f"""Analyze this job description and extract information in JSON format:

Job Description:
{jd_text}

Return a JSON object with:
- required_skills: list of must-have technical skills
- preferred_skills: list of nice-to-have skills
- soft_skills: list of soft skills needed
- experience_years: number (e.g., 5)
- education: required degree
- responsibilities: list of key responsibilities (top 5)
- keywords: list of important keywords for ATS
- seniority_level: junior/mid/senior/lead
- company_culture: brief description of culture/values mentioned

Only return valid JSON, no other text."""

        try:
            message = self.client.messages.create(
                model="claude-sonnet-4-5-20250929",
                max_tokens=1024,
                messages=[{"role": "user", "content": prompt}]
            )

            result = json.loads(message.content[0].text)
            return result
        except Exception as e:
            print(f"AI parsing error: {e}")
            return None


In [3]:
class ResumeGenerator:
    def __init__(self, user_profile, jd_analysis):
        self.profile = user_profile
        self.jd = jd_analysis

    def sanitize_text(self, text):
        """Remove Unicode characters that can't be encoded in latin-1"""
        # Replace common Unicode characters with ASCII equivalents
        replacements = {
            '\u2022': '-',  # bullet point
            '\u2013': '-',  # en dash
            '\u2014': '--', # em dash
            '\u2018': "'",  # left single quote
            '\u2019': "'",  # right single quote
            '\u201C': '"',  # left double quote
            '\u201D': '"',  # right double quote
            '\u2026': '...' # ellipsis
        }
        for unicode_char, ascii_char in replacements.items():
            text = text.replace(unicode_char, ascii_char)
        # Remove any remaining non-latin-1 characters
        return text.encode('latin-1', errors='ignore').decode('latin-1')

    def match_content(self):
        """Match user profile to JD requirements"""
        matched = {
            'skills': [],
            'experience': [],
            'projects': [],
            'match_score': 0
        }

        # Match skills
        required_skills = set([s.lower() for s in self.jd.get('required_skills', [])])
        user_skills = set([s.lower() for s in self.profile.get('skills', [])])
        matched['skills'] = list(required_skills.intersection(user_skills))

        # Calculate match score
        if required_skills:
            matched['match_score'] = int((len(matched['skills']) / len(required_skills)) * 100)

        # Match experience (simple filter by keywords)
        jd_keywords = [k.lower() for k in self.jd.get('keywords', [])]
        for exp in self.profile.get('experience', []):
            exp_text = f"{exp['role']} {exp['company']} {' '.join(exp['bullets'])}".lower()
            if any(keyword in exp_text for keyword in jd_keywords):
                matched['experience'].append(exp)

        # Match projects
        for project in self.profile.get('projects', []):
            project_text = f"{project['title']} {project['description']} {' '.join(project['technologies'])}".lower()
            if any(keyword in project_text for keyword in jd_keywords):
                matched['projects'].append(project)

        return matched

    def generate_pdf(self, filename="tailored_resume.pdf"):
        """Generate PDF resume"""
        if not PDF_AVAILABLE:
            print("⚠️ PDF generation not available")
            return None

        matched = self.match_content()

        pdf = FPDF()
        pdf.add_page()

        # Header
        pdf.set_font('Arial', 'B', 20)
        pdf.cell(0, 10, self.sanitize_text(self.profile['name']), ln=True, align='C')

        pdf.set_font('Arial', '', 10)
        contact = f"{self.profile['email']} | {self.profile['phone']} | {self.profile['location']}"
        pdf.cell(0, 6, self.sanitize_text(contact), ln=True, align='C')
        pdf.ln(5)

        # Match Score
        pdf.set_font('Arial', 'B', 12)
        pdf.set_text_color(0, 128, 0)
        pdf.cell(0, 8, f"Match Score: {matched['match_score']}%", ln=True)
        pdf.set_text_color(0, 0, 0)
        pdf.ln(3)

        # Professional Summary
        pdf.set_font('Arial', 'B', 14)
        pdf.cell(0, 8, 'PROFESSIONAL SUMMARY', ln=True)
        pdf.line(10, pdf.get_y(), 200, pdf.get_y())
        pdf.ln(2)

        pdf.set_font('Arial', '', 10)
        pdf.multi_cell(0, 5, self.sanitize_text(self.profile.get('summary', '')))
        pdf.ln(3)

        # Skills (matched first)
        pdf.set_font('Arial', 'B', 14)
        pdf.cell(0, 8, 'TECHNICAL SKILLS', ln=True)
        pdf.line(10, pdf.get_y(), 200, pdf.get_y())
        pdf.ln(2)

        pdf.set_font('Arial', '', 10)
        # Highlight matched skills
        matched_skills_text = "Matched Skills: " + ", ".join(matched['skills'])
        pdf.multi_cell(0, 5, self.sanitize_text(matched_skills_text))

        all_skills = ", ".join(self.profile.get('skills', []))
        pdf.multi_cell(0, 5, self.sanitize_text(f"All Skills: {all_skills}"))
        pdf.ln(3)

        # Experience
        pdf.set_font('Arial', 'B', 14)
        pdf.cell(0, 8, 'PROFESSIONAL EXPERIENCE', ln=True)
        pdf.line(10, pdf.get_y(), 200, pdf.get_y())
        pdf.ln(2)

        # Prioritize matched experience
        all_exp = matched['experience'] + [e for e in self.profile.get('experience', []) if e not in matched['experience']]

        for exp in all_exp[:3]:  # Top 3 experiences
            pdf.set_font('Arial', 'B', 11)
            pdf.cell(0, 6, self.sanitize_text(f"{exp['role']} - {exp['company']}"), ln=True)
            pdf.set_font('Arial', 'I', 9)
            pdf.cell(0, 5, self.sanitize_text(exp['dates']), ln=True)
            pdf.set_font('Arial', '', 9)
            for bullet in exp['bullets']:
                pdf.multi_cell(0, 5, self.sanitize_text(f"  - {bullet}"))
            pdf.ln(2)

        # Projects
        if matched['projects']:
            pdf.set_font('Arial', 'B', 14)
            pdf.cell(0, 8, 'RELEVANT PROJECTS', ln=True)
            pdf.line(10, pdf.get_y(), 200, pdf.get_y())
            pdf.ln(2)

            for project in matched['projects'][:2]:  # Top 2 projects
                pdf.set_font('Arial', 'B', 11)
                pdf.cell(0, 6, self.sanitize_text(project['title']), ln=True)
                pdf.set_font('Arial', '', 9)
                pdf.multi_cell(0, 5, self.sanitize_text(project['description']))
                pdf.set_font('Arial', 'I', 9)
                pdf.multi_cell(0, 5, self.sanitize_text(f"Technologies: {', '.join(project['technologies'])}"))
                pdf.ln(2)

        # Education
        pdf.set_font('Arial', 'B', 14)
        pdf.cell(0, 8, 'EDUCATION', ln=True)
        pdf.line(10, pdf.get_y(), 200, pdf.get_y())
        pdf.ln(2)

        for edu in self.profile.get('education', []):
            pdf.set_font('Arial', 'B', 11)
            pdf.cell(0, 6, self.sanitize_text(f"{edu['degree']} - {edu['school']}"), ln=True)
            pdf.set_font('Arial', '', 9)
            pdf.cell(0, 5, self.sanitize_text(edu['year']), ln=True)
            pdf.ln(2)

        # Save
        pdf.output(filename)
        print(f"✅ PDF Resume generated: {filename}")
        return filename

    def generate_docx(self, filename="tailored_resume.docx"):
        """Generate Word document resume"""
        if not DOCX_AVAILABLE:
            print("⚠️ DOCX generation not available")
            return None

        matched = self.match_content()

        doc = Document()

        # Header
        header = doc.add_paragraph()
        header.alignment = WD_ALIGN_PARAGRAPH.CENTER
        name = header.add_run(self.profile['name'])
        name.font.size = Pt(20)
        name.font.bold = True

        contact = doc.add_paragraph()
        contact.alignment = WD_ALIGN_PARAGRAPH.CENTER
        contact_text = contact.add_run(f"{self.profile['email']} | {self.profile['phone']} | {self.profile['location']}")
        contact_text.font.size = Pt(10)

        # Match Score
        match_para = doc.add_paragraph()
        match_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
        match_run = match_para.add_run(f"JD Match Score: {matched['match_score']}%")
        match_run.font.size = Pt(12)
        match_run.font.bold = True
        match_run.font.color.rgb = RGBColor(0, 128, 0)

        # Summary
        doc.add_heading('PROFESSIONAL SUMMARY', level=1)
        doc.add_paragraph(self.profile.get('summary', ''))

        # Skills
        doc.add_heading('TECHNICAL SKILLS', level=1)
        doc.add_paragraph(f"Matched Skills: {', '.join(matched['skills'])}", style='List Bullet')
        doc.add_paragraph(f"All Skills: {', '.join(self.profile.get('skills', []))}")

        # Experience
        doc.add_heading('PROFESSIONAL EXPERIENCE', level=1)
        all_exp = matched['experience'] + [e for e in self.profile.get('experience', []) if e not in matched['experience']]

        for exp in all_exp[:3]:
            exp_header = doc.add_paragraph()
            exp_role = exp_header.add_run(f"{exp['role']} - {exp['company']}")
            exp_role.font.bold = True
            exp_role.font.size = Pt(11)

            dates = doc.add_paragraph(exp['dates'])
            dates.runs[0].font.italic = True

            for bullet in exp['bullets']:
                doc.add_paragraph(bullet, style='List Bullet')

        # Projects
        if matched['projects']:
            doc.add_heading('RELEVANT PROJECTS', level=1)
            for project in matched['projects'][:2]:
                project_header = doc.add_paragraph()
                proj_title = project_header.add_run(project['title'])
                proj_title.font.bold = True

                doc.add_paragraph(project['description'])
                doc.add_paragraph(f"Technologies: {', '.join(project['technologies'])}")

        # Education
        doc.add_heading('EDUCATION', level=1)
        for edu in self.profile.get('education', []):
            edu_para = doc.add_paragraph()
            edu_text = edu_para.add_run(f"{edu['degree']} - {edu['school']}")
            edu_text.font.bold = True
            doc.add_paragraph(edu['year'])

        # Save
        doc.save(filename)
        print(f"✅ Word Resume generated: {filename}")
        return filename

In [4]:
def generate_docx(self, filename="tailored_resume.docx"):
        """Generate Word document resume"""
        if not DOCX_AVAILABLE:
            print("⚠️ DOCX generation not available")
            return None

        matched = self.match_content()

        doc = Document()

        # Header
        header = doc.add_paragraph()
        header.alignment = WD_ALIGN_PARAGRAPH.CENTER
        name = header.add_run(self.profile['name'])
        name.font.size = Pt(20)
        name.font.bold = True

        contact = doc.add_paragraph()
        contact.alignment = WD_ALIGN_PARAGRAPH.CENTER
        contact_text = contact.add_run(f"{self.profile['email']} | {self.profile['phone']} | {self.profile['location']}")
        contact_text.font.size = Pt(10)

        # Match Score
        match_para = doc.add_paragraph()
        match_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
        match_run = match_para.add_run(f"JD Match Score: {matched['match_score']}%")
        match_run.font.size = Pt(12)
        match_run.font.bold = True
        match_run.font.color.rgb = RGBColor(0, 128, 0)

        # Summary
        doc.add_heading('PROFESSIONAL SUMMARY', level=1)
        doc.add_paragraph(self.profile.get('summary', ''))

        # Skills
        doc.add_heading('TECHNICAL SKILLS', level=1)
        doc.add_paragraph(f"Matched Skills: {', '.join(matched['skills'])}", style='List Bullet')
        doc.add_paragraph(f"All Skills: {', '.join(self.profile.get('skills', []))}")

        # Experience
        doc.add_heading('PROFESSIONAL EXPERIENCE', level=1)
        all_exp = matched['experience'] + [e for e in self.profile.get('experience', []) if e not in matched['experience']]

        for exp in all_exp[:3]:
            exp_header = doc.add_paragraph()
            exp_role = exp_header.add_run(f"{exp['role']} - {exp['company']}")
            exp_role.font.bold = True
            exp_role.font.size = Pt(11)

            dates = doc.add_paragraph(exp['dates'])
            dates.runs[0].font.italic = True

            for bullet in exp['bullets']:
                doc.add_paragraph(bullet, style='List Bullet')

        # Projects
        if matched['projects']:
            doc.add_heading('RELEVANT PROJECTS', level=1)
            for project in matched['projects'][:2]:
                project_header = doc.add_paragraph()
                proj_title = project_header.add_run(project['title'])
                proj_title.font.bold = True

                doc.add_paragraph(project['description'])
                doc.add_paragraph(f"Technologies: {', '.join(project['technologies'])}")

        # Education
        doc.add_heading('EDUCATION', level=1)
        for edu in self.profile.get('education', []):
            edu_para = doc.add_paragraph()
            edu_text = edu_para.add_run(f"{edu['degree']} - {edu['school']}")
            edu_text.font.bold = True
            doc.add_paragraph(edu['year'])

        # Save
        doc.save(filename)
        print(f"✅ Word Resume generated: {filename}")
        return filename

In [5]:
# ============= EXAMPLE USAGE =============

# Sample user profile (your master resume data)
user_profile = {
    'name': 'John Doe',
    'email': 'john.doe@email.com',
    'phone': '(555) 123-4567',
    'location': 'San Francisco, CA',
    'summary': 'Experienced Full Stack Developer with 5+ years building scalable web applications. Passionate about clean code and user experience.',
    'skills': [
        'Python', 'JavaScript', 'React', 'Node.js', 'PostgreSQL',
        'MongoDB', 'Docker', 'AWS', 'Git', 'REST API', 'SQL',
        'HTML', 'CSS', 'Express', 'Flask', 'Agile'
    ],
    'experience': [
        {
            'role': 'Senior Software Engineer',
            'company': 'Tech Corp',
            'dates': 'Jan 2021 - Present',
            'bullets': [
                'Built scalable microservices using Python Flask and Docker, serving 1M+ users',
                'Led frontend development with React and TypeScript, improving load time by 40%',
                'Implemented CI/CD pipelines using Jenkins and AWS, reducing deployment time by 60%'
            ]
        },
        {
            'role': 'Software Developer',
            'company': 'StartupXYZ',
            'dates': 'Jun 2019 - Dec 2020',
            'bullets': [
                'Developed REST APIs using Node.js and Express for e-commerce platform',
                'Designed PostgreSQL database schemas handling 100K+ daily transactions',
                'Collaborated with cross-functional teams in Agile environment'
            ]
        },
        {
            'role': 'Junior Developer',
            'company': 'Web Solutions Inc',
            'dates': 'Jan 2018 - May 2019',
            'bullets': [
                'Created responsive web applications using HTML, CSS, and JavaScript',
                'Maintained MongoDB databases and optimized queries',
                'Participated in code reviews and pair programming sessions'
            ]
        }
    ],
    'projects': [
        {
            'title': 'E-commerce Analytics Dashboard',
            'description': 'Built real-time analytics dashboard for tracking sales metrics and customer behavior',
            'technologies': ['React', 'Python', 'PostgreSQL', 'AWS', 'Docker']
        },
        {
            'title': 'Task Management API',
            'description': 'RESTful API for task management with authentication and real-time updates',
            'technologies': ['Node.js', 'Express', 'MongoDB', 'Socket.io']
        },
        {
            'title': 'Machine Learning Price Predictor',
            'description': 'ML model to predict housing prices using regression algorithms',
            'technologies': ['Python', 'Pandas', 'Scikit-learn', 'Flask']
        }
    ],
    'education': [
        {
            'degree': 'B.S. Computer Science',
            'school': 'University of California',
            'year': '2017'
        }
    ]
}

# Sample job description
sample_jd = """
Senior Full Stack Engineer

We are seeking an experienced Full Stack Engineer to join our growing team.

Responsibilities:
- Design and develop web applications using React and Node.js
- Build scalable APIs and microservices
- Work with PostgreSQL databases
- Deploy applications on AWS infrastructure
- Collaborate with product team in Agile environment

Requirements:
- 5+ years of software development experience
- Strong proficiency in JavaScript, Python, and SQL
- Experience with React and Node.js
- Knowledge of Docker and AWS
- Bachelor's degree in Computer Science
- Excellent problem-solving and communication skills

Nice to have:
- Experience with microservices architecture
- CI/CD pipeline experience
- Knowledge of MongoDB
"""

# Basic parsing (without AI)
print("=" * 60)
print("STEP 1: PARSING JOB DESCRIPTION")
print("=" * 60)

# For AI parsing, uncomment and add your API key:
# api_key = "your-anthropic-api-key"
# ai_parser = AIJobParser(api_key=api_key)
# jd_analysis = ai_parser.parse_with_ai(sample_jd)

# Manual JD analysis for demo (replace with AI parsing above)
jd_analysis = {
    'required_skills': ['JavaScript', 'Python', 'SQL', 'React', 'Node.js', 'PostgreSQL', 'Docker', 'AWS'],
    'preferred_skills': ['MongoDB', 'Microservices', 'CI/CD'],
    'soft_skills': ['problem-solving', 'communication', 'collaboration'],
    'experience_years': 5,
    'education': 'Bachelor in Computer Science',
    'responsibilities': [
        'Design web applications',
        'Build scalable APIs',
        'Work with databases',
        'Deploy on AWS',
        'Collaborate in Agile'
    ],
    'keywords': ['react', 'node.js', 'postgresql', 'aws', 'docker', 'agile', 'api', 'microservices'],
    'seniority_level': 'senior'
}

print("\n✅ JD Analysis Complete!")
print(f"Required Skills: {', '.join(jd_analysis['required_skills'])}")
print(f"Experience Required: {jd_analysis['experience_years']} years")
print(f"Seniority: {jd_analysis['seniority_level']}")

# Generate tailored resume
print("\n" + "=" * 60)
print("STEP 2: GENERATING TAILORED RESUME")
print("=" * 60)

generator = ResumeGenerator(user_profile, jd_analysis)

# Show match score
matched = generator.match_content()
print(f"\n📊 Your Match Score: {matched['match_score']}%")
print(f"✅ Matched Skills: {', '.join(matched['skills'])}")
print(f"📁 Relevant Experience Entries: {len(matched['experience'])}")
print(f"🚀 Relevant Projects: {len(matched['projects'])}")

# Generate resumes
print("\n" + "=" * 60)
print("STEP 3: CREATING DOWNLOADABLE RESUMES")
print("=" * 60)

pdf_file = generator.generate_pdf("my_tailored_resume.pdf")
docx_file = generator.generate_docx("my_tailored_resume.docx")

print("\n✨ Resume generation complete!")
print("\nDownload your tailored resumes:")
print(f"  📄 PDF: {pdf_file}")
print(f"  📝 Word: {docx_file}")

# To download in Colab:
print("\n💡 To download in Colab, run:")
print("from google.colab import files")
if pdf_file:
    print(f"files.download('{pdf_file}')")
if docx_file:
    print(f"files.download('{docx_file}')")

STEP 1: PARSING JOB DESCRIPTION

✅ JD Analysis Complete!
Required Skills: JavaScript, Python, SQL, React, Node.js, PostgreSQL, Docker, AWS
Experience Required: 5 years
Seniority: senior

STEP 2: GENERATING TAILORED RESUME

📊 Your Match Score: 100%
✅ Matched Skills: javascript, python, postgresql, docker, node.js, react, aws, sql
📁 Relevant Experience Entries: 2
🚀 Relevant Projects: 2

STEP 3: CREATING DOWNLOADABLE RESUMES
✅ PDF Resume generated: my_tailored_resume.pdf
✅ Word Resume generated: my_tailored_resume.docx

✨ Resume generation complete!

Download your tailored resumes:
  📄 PDF: my_tailored_resume.pdf
  📝 Word: my_tailored_resume.docx

💡 To download in Colab, run:
from google.colab import files
files.download('my_tailored_resume.pdf')
files.download('my_tailored_resume.docx')
